In [1]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
from tkinter.simpledialog import askfloat
from tkinter import ttk
from PIL import Image, ImageTk
import datetime  # To manage timestamps in the credit history

# Utility function to create the SQLite database with required tables
def create_database():
    conn = sqlite3.connect('mess_system.db')
    c = conn.cursor()

    # Create table for students with an additional field for credit history
    c.execute('''CREATE TABLE IF NOT EXISTS students (
                    student_id TEXT PRIMARY KEY,
                    password TEXT,
                    credits REAL DEFAULT 0.0
                 )''')

    # Create table for check-in/out logs
    c.execute('''CREATE TABLE IF NOT EXISTS check_in_out (
                    student_id TEXT,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    status TEXT
                 )''')

    # Create table for credit history to track credit changes
    c.execute('''CREATE TABLE IF NOT EXISTS credit_history (
                    student_id TEXT,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    change REAL,
                    reason TEXT
                 )''')

    conn.commit()
    conn.close()


class MessSystemApp(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Giki Mess System")
        self.geometry("600x450")

        # Set a consistent color theme
        self.bg_color = "#e0f7fa"  # Light cyan background
        self.fg_color = "#006064"  # Dark teal for text
        self.highlight_color = "#009688"  # A lighter teal for buttons

        self.configure(bg=self.bg_color)

        self.student_id = None

        self.header_frame = tk.Frame(self, bg=self.bg_color)
        self.header_frame.pack(pady=20)

        # Adding frames for better organization
        tk.Label(self.header_frame, text="Giki Mess System", font=("Helvetica", 22, "bold"), fg=self.fg_color, bg=self.bg_color).pack()

        self.main_frame = tk.Frame(self, bg=self.bg_color)
        self.main_frame.pack(expand=True, fill=tk.BOTH)

        self.show_login_page()

    # Function to clear all widgets in the main frame
    def clear_frame(self):
        for widget in self.main_frame.winfo_children():
            widget.destroy()

    # Function to display the login page
    def show_login_page(self):
        self.clear_frame()

        tk.Label(self.main_frame, text="Student ID:", font=("Helvetica", 12), fg=self.fg_color, bg=self.bg_color).grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)
        self.student_id_entry = tk.Entry(self.main_frame)
        self.student_id_entry.grid(row=0, column=1, padx=10, pady=10)

        tk.Label(self.main_frame, text="Password:", font=("Helvetica", 12), fg=self.fg_color, bg=self.bg_color).grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)
        self.password_entry = tk.Entry(self.main_frame, show='*')
        self.password_entry.grid(row=1, column=1, padx=10, pady=10)

        login_button = tk.Button(self.main_frame, text="Login", font=("Helvetica", 12), command=self.authenticate, bg=self.highlight_color, fg="white")
        login_button.grid(row=2, column=0, columnspan=2, pady=10)

        signup_button = tk.Button(self.main_frame, text="Signup", font=("Helvetica", 12), command=self.show_signup_page, bg="#f57c00", fg="white")
        signup_button.grid(row=3, column=0, columnspan=2, pady=10)

    # Function to handle login authentication
    def authenticate(self):
        student_id = self.student_id_entry.get()
        password = self.password_entry.get()

        conn = sqlite3.connect('mess_system.db')
        c = conn.cursor()

        c.execute("SELECT * FROM students WHERE student_id = ? AND password = ?", (student_id, password))
        user = c.fetchone()

        conn.close()

        if user:
            self.student_id = student_id
            self.show_main_menu()
        else:
            messagebox.showerror("Login Failed", "Invalid Student ID or Password")

    # Function to display the main menu
    def show_main_menu(self):
        self.clear_frame()

        tk.Label(self.main_frame, text=f"Welcome to Giki Mess System, {self.student_id}!", font=("Helvetica", 14), fg=self.fg_color, bg=self.bg_color).pack(pady=10)

        credits_button = tk.Button(self.main_frame, text="Credits", font=("Helvetica", 12), command=self.show_credits_page, bg=self.highlight_color, fg="white")
        credits_button.pack(pady=10)

        in_out_button = tk.Button(self.main_frame, text="Check In/Out", font=("Helvetica", 12), command=self.show_check_in_out_page, bg=self.highlight_color, fg="white")
        in_out_button.pack(pady=10)

        menu_button = tk.Button(self.main_frame, text="Menu", font=("Helvetica", 12), command=self.show_menu_page, bg=self.highlight_color, fg="white")
        menu_button.pack(pady=10)

        logout_button = tk.Button(self.main_frame, text="Logout", font=("Helvetica", 12), command=self.show_login_page, bg="#f57c00", fg="white")
        logout_button.pack(pady=10)

    # Function to display the signup page
    def show_signup_page(self):
        self.clear_frame()

        tk.Label(self.main_frame, text="New Student Signup", font=("Helvetica", 14), fg=self.fg_color, bg=self.bg_color).grid(row=0, column=0, columnspan=2, pady=10)

        tk.Label(self.main_frame, text="New Student ID:", font=("Helvetica", 12), fg=self.fg_color, bg=self.bg_color).grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)
        self.new_student_id_entry = tk.Entry(self.main_frame)
        self.new_student_id_entry.grid(row=1, column=1, padx=10, pady=10)

        tk.Label(self.main_frame, text="New Password:", font=("Helvetica", 12), fg=self.fg_color, bg=self.bg_color).grid(row=2, column=0, padx=10, pady=10, sticky=tk.W)
        self.new_password_entry = tk.Entry(self.main_frame, show='*')
        self.new_password_entry.grid(row=2, column=1, padx=10, pady=10)

        signup_button = tk.Button(self.main_frame, text="Signup", font=("Helvetica", 12), command=self.signup, bg=self.highlight_color, fg="white")
        signup_button.grid(row=3, column=0, columnspan=2, pady=10)

        back_button = tk.Button(self.main_frame, text="Back to Login", font=("Helvetica", 12), command=self.show_login_page, bg=self.highlight_color, fg="white")
        back_button.grid(row=4, column=0, columnspan=2, pady=10)

    # Function to handle new student signup
    def signup(self):
        new_student_id = self.new_student_id_entry.get()
        new_password = self.new_password_entry.get()

        if not new_student_id or not new_password:
            messagebox.showerror("Error", "Please enter a Student ID and Password")
            return

        conn = sqlite3.connect('mess_system.db')
        c = conn.cursor()

        # Check if the student ID already exists
        c.execute("SELECT * FROM students WHERE student_id = ?", (new_student_id,))
        if c.fetchone() is not None:
            conn.close()
            messagebox.showerror("Error", "Student ID already exists. Choose a different one.")
            return

        # Insert the new student into the database
        c.execute("INSERT INTO students (student_id, password, credits) VALUES (?, ?, ?)", (new_student_id, new_password, 10.0))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", "Signup successful! You can now log in.")

        self.show_login_page()

    # Function to display the credits page
    def show_credits_page(self):
        self.clear_frame()

        conn = sqlite3.connect('mess_system.db')
        c = conn.cursor()

        c.execute("SELECT credits FROM students WHERE student_id = ?", (self.student_id,))
        credits = c.fetchone()[0]

        tk.Label(self.main_frame, text=f"Remaining Credits: {credits}", font=("Helvetica", 14), fg=self.fg_color, bg=self.bg_color).pack(pady=10)

        # Load credits or buy packages
        load_credits_button = tk.Button(self.main_frame, text="Load/Buy Credits", font=("Helvetica", 12), command=self.load_credits, bg=self.highlight_color, fg="white")
        load_credits_button.pack(pady=10)

        view_history_button = tk.Button(self.main_frame, text="View Credit History", font=("Helvetica", 12), command=self.view_credit_history, bg=self.highlight_color, fg="white")
        view_history_button.pack(pady=10)

        back_button = tk.Button(self.main_frame, text="Back to Main Menu", font=("Helvetica", 12), command=self.show_main_menu, bg=self.highlight_color, fg="white")
        back_button.pack(pady=10)

        conn.close()

    # Function to load credits into the student's account
    def load_credits(self):
        new_credits = askfloat("Load Credits", "Enter amount of credits to load:")

        if new_credits is not None:
            conn = sqlite3.connect('mess_system.db')
            c = conn.cursor()

            # Update the credits for the student and record in credit history
            c.execute("UPDATE students SET credits = credits + ? WHERE student_id = ?", (new_credits, self.student_id))
            c.execute("INSERT INTO credit_history (student_id, change, reason) VALUES (?, ?, ?)", (self.student_id, new_credits, "Credits Purchased"))

            conn.commit()
            conn.close()

            messagebox.showinfo("Success", "Credits loaded successfully!")

            self.show_credits_page()

    # Function to view the credit history
    def view_credit_history(self):
        self.clear_frame()

        conn = sqlite3.connect('mess_system.db')
        c = conn.cursor()

        # Get the credit history for the student
        c.execute("SELECT timestamp, change, reason FROM credit_history WHERE student_id = ? ORDER BY timestamp DESC", (self.student_id,))

        history = c.fetchall()

        tk.Label(self.main_frame, text="Credit History", font=("Helvetica", 14), fg=self.fg_color, bg=self.bg_color).pack(pady=10)

        for record in history:
            timestamp, change, reason = record
            tk.Label(self.main_frame, text=f"{timestamp}: {change:.2f} credits - {reason}", font=("Helvetica", 12), fg=self.fg_color, bg=self.bg_color).pack(pady=5)

        back_button = tk.Button(self.main_frame, text="Back to Credits Page", font=("Helvetica", 12), command=self.show_credits_page, bg=self.highlight_color, fg="white")
        back_button.pack(pady=10)

        conn.close()

    # Function to display the check-in/out page
    def show_check_in_out_page(self):
        self.clear_frame()

        conn = sqlite3.connect('mess_system.db')
        c = conn.cursor()

        # Get the last status
        c.execute("SELECT status FROM check_in_out WHERE student_id = ? ORDER BY timestamp DESC LIMIT 1",
                  (self.student_id,))
        last_status = c.fetchone()

        # Determine new status and update credits if necessary
        if last_status is None or last_status[0] == "Out":
            new_status = "In"
            credit_deduction = 1.0  # Deduct 1 credit when checking in
        else:
            new_status = "Out"
            credit_deduction = 0.0  # No deduction when checking out

        # Check if student has enough credits to check in
        if new_status == "In":
            c.execute("SELECT credits FROM students WHERE student_id = ?", (self.student_id,))
            current_credits = c.fetchone()[0]

            if current_credits < credit_deduction:
                conn.close()
                messagebox.showerror("Error", "Not enough credits to check in.")
                return

            # Deduct credits and update credit history
            c.execute("UPDATE students SET credits = credits - ? WHERE student_id = ?", (credit_deduction, self.student_id))
            c.execute("INSERT INTO credit_history (student_id, change, reason) VALUES (?, ?, ?)", (self.student_id, -credit_deduction, "Check-in Deduction"))

        # Insert new check-in/out record
        c.execute("INSERT INTO check_in_out (student_id, status) VALUES (?, ?)", (self.student_id, new_status))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", f"Checked {new_status} successfully!")

        back_button = tk.Button(self.main_frame, text="Back to Main Menu", font=("Helvetica", 12), command=self.show_main_menu, bg=self.highlight_color, fg="white")
        back_button.pack(pady=10)

    # Function to display the weekly menu
    def show_menu_page(self):
        self.clear_frame()

        # Example menu data
        menu = [
            "Monday: Cheese Omlete, Haleem, Mix Vegetables",
            "Tuesday: Omlete, Lobiya, Savour Rice",
            "Wednesday: Half Fry, Fried Rice, Achari Karahi",
            "Thursday: Aloo Paratha, White Karahi, Kabali Pulao",
            "Friday: French Toast, Brown Rice, Chicken Shashlik",
            "Saturday: Halwa puri, Macroni, Chicken Karahi",
            "Sunday: Chicken Spread, White Rice with chickpeas, Beef pulao "
        ]

        tk.Label(self.main_frame, text="Weekly Menu", font=("Helvetica", 14), fg=self.fg_color, bg=self.bg_color).pack(pady=10)

        for item in menu:
            tk.Label(self.main_frame, text=item, font=("Helvetica", 12), fg=self.fg_color, bg=self.bg_color).pack(pady=5)

        back_button = tk.Button(self.main_frame, text="Back to Main Menu", font=("Helvetica", 12), command=self.show_main_menu, bg=self.highlight_color, fg="white")
        back_button.pack(pady=10)


create_database()

app = MessSystemApp()
app.mainloop()
